In [1]:
from pyspark.sql import SparkSession

In [3]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [5]:
spark = SparkSession.builder.appName('data_processing').getOrCreate()

In [4]:
schema = StructType().add('user_id',"string").add("country","string").add("browser","string").add("OS","string").add("age","integer")

In [6]:
df_custom = spark.createDataFrame([("A203",'India',"Chrome","WIN",33),("A201",'China',"Safari","MacOS",35),("A205",'UK',"Mozilla","Linux",25)],schema = schema)

In [8]:
df_custom.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- browser: string (nullable = true)
 |-- OS: string (nullable = true)
 |-- age: integer (nullable = true)



In [ ]:
df_custom.show()